In [94]:
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import IPython.display as ipd

In [95]:
train = pd.read_csv("train.csv")

In [96]:
train.head()

,Name,Label
0,C:\Users\turok1997\Desktop\project_junction\vo...,Stopped
1,C:\Users\turok1997\Desktop\project_junction\vo...,Stopped
2,C:\Users\turok1997\Desktop\project_junction\vo...,Stopped
3,C:\Users\turok1997\Desktop\project_junction\vo...,Stopped
4,C:\Users\turok1997\Desktop\project_junction\vo...,Stopped


In [99]:
train = train.sample(frac=1)

In [100]:
train.head()

,Name,Label
142,C:\Users\turok1997\Desktop\project_junction\vo...,Good
221,C:\Users\turok1997\Desktop\project_junction\vo...,Good
309,C:\Users\turok1997\Desktop\project_junction\vo...,Stopped
131,C:\Users\turok1997\Desktop\project_junction\vo...,Good
159,C:\Users\turok1997\Desktop\project_junction\vo...,Good


In [101]:
def parser(row):
    # get the auidio files by id 
    filename = row[0]
    try:
        # use kaise_fast to get faster feature extraction
        X, samplerate = librosa.load(filename, res_type = 'kaiser_fast')
        #extract mfcc feature
        mfcc = np.mean(librosa.feature.mfcc(y = X, sr = samplerate, n_mfcc = 40).T, axis=0)
    except Exception as e:
        print(e)
        return None, None
    feature = mfcc
    label = row[1]
    return [feature, label]

In [102]:
data = train.apply(parser, axis=1)

In [105]:
data.to_csv("preprocessed.csv")

In [106]:
audio = pd.DataFrame(columns=['feature','labels'])

In [107]:
for d in data:
    audio = audio.append({'feature':d[0], 'labels':d[1]}, ignore_index=True)


In [108]:
audio.head()

,feature,labels
0,"[-95.25659088856045, 87.88587888379766, -14.76...",Good
1,"[-96.63717659333744, 95.1054511305555, -31.897...",Good
2,"[-126.5541062129846, 95.80420699844338, -13.66...",Stopped
3,"[-100.49741056817338, 94.760363678197, -9.7901...",Good
4,"[-65.91828079731668, 84.19437067544071, -30.04...",Good


In [147]:
# one hot encoding
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
X = np.array(audio['feature'].tolist())
y = np.array(audio['labels'].tolist())
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(y))

Stopped


In [113]:
train_X = X[:300]
train_y = y[:300]
val_x = X[300:lines]
val_y = y[300:lines]

In [117]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from sklearn import metrics 

num_labels = y.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [135]:
model.fit(train_X, train_y, batch_size=16, epochs=60, validation_data=(val_x, val_y))

Train on 300 samples, validate on 70 samples
Epoch 1/60
300/300 [==============================] - 0s 351us/step - loss: 0.3631 - acc: 0.8533 - val_loss: 0.5865 - val_acc: 0.8000
Epoch 2/60
300/300 [==============================] - 0s 575us/step - loss: 0.3675 - acc: 0.8367 - val_loss: 0.6379 - val_acc: 0.8143
Epoch 3/60
300/300 [==============================] - 0s 579us/step - loss: 0.4084 - acc: 0.8333 - val_loss: 0.5386 - val_acc: 0.8143
Epoch 4/60
300/300 [==============================] - 0s 575us/step - loss: 0.3728 - acc: 0.8433 - val_loss: 0.5562 - val_acc: 0.8143
Epoch 5/60
300/300 [==============================] - 0s 520us/step - loss: 0.3473 - acc: 0.8633 - val_loss: 0.5522 - val_acc: 0.8000
Epoch 6/60
300/300 [==============================] - 0s 508us/step - loss: 0.3465 - acc: 0.8933 - val_loss: 0.5925 - val_acc: 0.8429
Epoch 7/60
300/300 [==============================] - 0s 544us/step - loss: 0.3595 - acc: 0.8500 - val_loss: 0.6121 - val_acc: 0.7857
Epoch 8/60
300/30

In [142]:
predicted_prob = model.predict_proba(val_x)
predicted_prob

array([[5.95467746e-01, 3.91625106e-01, 1.29071185e-02],
       [2.11471710e-02, 9.70337927e-01, 8.51492677e-03],
       [2.09342329e-06, 8.12583268e-01, 1.87414601e-01],
       [1.60739794e-01, 8.32646251e-01, 6.61397260e-03],
       [1.73551202e-01, 7.35911310e-01, 9.05374959e-02],
       [9.38316137e-02, 8.25692475e-01, 8.04758146e-02],
       [6.15219542e-05, 5.27863428e-02, 9.47152138e-01],
       [1.13487683e-01, 8.02030504e-01, 8.44818130e-02],
       [3.27059883e-04, 6.97657585e-01, 3.02015364e-01],
       [1.76199377e-02, 8.53923678e-01, 1.28456473e-01],
       [1.61568874e-08, 1.59851729e-03, 9.98401463e-01],
       [4.44890389e-07, 9.95946348e-01, 4.05313633e-03],
       [2.40397230e-02, 9.71940577e-01, 4.01973724e-03],
       [1.44773794e-05, 8.45810175e-02, 9.15404558e-01],
       [4.14986210e-03, 9.84271109e-01, 1.15790581e-02],
       [6.20317187e-10, 1.62790343e-01, 8.37209642e-01],
       [5.75494505e-02, 9.38264489e-01, 4.18613758e-03],
       [2.75333803e-02, 9.34548

In [202]:
predicted = np.argmax(predicted_prob, axis=1)

In [203]:
true_value = np.argmax(val_y, axis=1)

In [204]:
from sklearn.metrics import confusion_matrix

confusion_matrix(true_value, predicted)

array([[ 7,  1,  0],
       [ 0, 40,  2],
       [ 0,  6, 14]], dtype=int64)

In [173]:
#example file which transmit from good to failure state
file = "C:/Users/turok1997/Desktop/project_junction/device1_channel1_20181021180537.wav"
def parser(row):
    # get the auidio files by id 
    filename = row
    try:
        # use kaise_fast to get faster feature extraction
        X, samplerate = librosa.load(filename, res_type = 'kaiser_fast')
        #extract mfcc feature
        mfcc = np.mean(librosa.feature.mfcc(y = X, sr = samplerate, n_mfcc = 40).T, axis=0)
    except Exception as e:
        print(e)
        return None, None
    feature = mfcc
    return feature

In [175]:
features = parser(file)

In [193]:
f = features.reshape(1,-1)

In [196]:
model.predict_proba(f)

array([[0.9191676 , 0.07845156, 0.00238094]], dtype=float32)